In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn import preprocessing
from dgl.data import DGLDataset
import dgl
import time
import networkx as nx
import category_encoders as ce
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import torch
import tqdm
import math

from typing import *
from sklearn.preprocessing import StandardScaler, Normalizer
import socket
import struct
import random
from sklearn.model_selection import train_test_split

/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_name = "NF-UNSW-NB15-v3.parquet"
# file_name = "final.csv"
data = pd.read_parquet(file_name)

In [3]:
data.Label.value_counts()

Label
0    2151027
1      91904
Name: count, dtype: int64

In [4]:
data.rename(columns=lambda x: x.strip(), inplace=True)
data['IPV4_SRC_ADDR'] = data["IPV4_SRC_ADDR"].apply(str)
data['L4_SRC_PORT'] = data["L4_SRC_PORT"].apply(str)
data['IPV4_DST_ADDR'] = data["IPV4_DST_ADDR"].apply(str)
data['L4_DST_PORT'] = data["L4_DST_PORT"].apply(str)

In [5]:
data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)

In [6]:
data.Attack.unique()

array(['Benign', 'Fuzzers', 'Exploits', 'Backdoor', 'Generic', 'DoS',
       'Reconnaissance', 'Shellcode', 'Analysis', 'Worms'], dtype=object)

In [ ]:
# Scale the dataset based on your needs (machine capacity) ideally ~500k rows

# data = data.groupby(by='Attack').sample(frac=0.1, random_state=13)
data_attack = data[data['Label'] == 1]
data_benign = data[data['Label'] == 0].sample(frac=0.02, random_state=13)
data = pd.concat([data_attack, data_benign], axis=0)
data = data.sample(frac=1, random_state=13).reset_index(drop=True)
data.Label.value_counts()

Label
1    91904
0    43021
Name: count, dtype: int64

In [8]:
data.groupby(by="Attack").count()

,FLOW_START_MILLISECONDS,FLOW_END_MILLISECONDS,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,...,FTP_COMMAND_RET_CODE,SRC_TO_DST_IAT_MIN,SRC_TO_DST_IAT_MAX,SRC_TO_DST_IAT_AVG,SRC_TO_DST_IAT_STDDEV,DST_TO_SRC_IAT_MIN,DST_TO_SRC_IAT_MAX,DST_TO_SRC_IAT_AVG,DST_TO_SRC_IAT_STDDEV,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Analysis,1226,1226,1226,1226,1226,1226,1226,1226,1226,1226,...,1226,1226,1226,1226,1226,1226,1226,1226,1226,1226
Backdoor,3451,3451,3451,3451,3451,3451,3451,3451,3451,3451,...,3451,3451,3451,3451,3451,3451,3451,3451,3451,3451
Benign,43021,43021,43021,43021,43021,43021,43021,43021,43021,43021,...,43021,43021,43021,43021,43021,43021,43021,43021,43021,43021
DoS,5050,5050,5050,5050,5050,5050,5050,5050,5050,5050,...,5050,5050,5050,5050,5050,5050,5050,5050,5050,5050
Exploits,38816,38816,38816,38816,38816,38816,38816,38816,38816,38816,...,38816,38816,38816,38816,38816,38816,38816,38816,38816,38816
Fuzzers,25588,25588,25588,25588,25588,25588,25588,25588,25588,25588,...,25588,25588,25588,25588,25588,25588,25588,25588,25588,25588
Generic,4760,4760,4760,4760,4760,4760,4760,4760,4760,4760,...,4760,4760,4760,4760,4760,4760,4760,4760,4760,4760
Reconnaissance,11291,11291,11291,11291,11291,11291,11291,11291,11291,11291,...,11291,11291,11291,11291,11291,11291,11291,11291,11291,11291
Shellcode,1586,1586,1586,1586,1586,1586,1586,1586,1586,1586,...,1586,1586,1586,1586,1586,1586,1586,1586,1586,1586


In [9]:
X = data.drop(columns=["Attack", "Label", "FLOW_START_MILLISECONDS", "FLOW_END_MILLISECONDS",
                       "SRC_TO_DST_IAT_MIN", "SRC_TO_DST_IAT_MAX", "SRC_TO_DST_IAT_AVG",
                       "SRC_TO_DST_IAT_STDDEV", "DST_TO_SRC_IAT_MIN", "DST_TO_SRC_IAT_MAX",
                       "DST_TO_SRC_IAT_AVG", "DST_TO_SRC_IAT_STDDEV"])
y = data[["Attack", "Label"]]

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=13, stratify=y)

In [10]:
encoder = ce.TargetEncoder(cols=['TCP_FLAGS','L7_PROTO','PROTOCOL',
                                  'CLIENT_TCP_FLAGS','SERVER_TCP_FLAGS','ICMP_TYPE',
                                  'ICMP_IPV4_TYPE','DNS_QUERY_ID','DNS_QUERY_TYPE',
                                  'FTP_COMMAND_RET_CODE'])
encoder.fit(X_train, y_train.Label)

# Transform on training set
X_train = encoder.transform(X_train)

# Transform on testing set
X_test = encoder.transform(X_test)

In [11]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [12]:
# (Modified)
scaler = Normalizer()
cols_to_norm = list(set(list(X_train.iloc[:, 2:].columns))) # Ignore first two as the represents IP addresses
scaler.fit(X_train[cols_to_norm])

# Transform on training set
X_train[cols_to_norm] = scaler.transform(X_train[cols_to_norm])
X_train['h'] = X_train.iloc[:, 2:].values.tolist()
X_train['id'] = X_train.index

# Transform on testing set
X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test.iloc[:, 2:].values.tolist()
X_test['id'] = X_test.index

train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [13]:
X_train.head()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,h,id
460141,175.45.176.1,149.171.126.14,0.000002,2.594677e-06,0.002873,0.000047,0.179076,0.000154,0.000001,0.000001,...,0.048430,0.048430,2.926896e-06,2.927499e-06,0.000002,0.000002,0.0,0.000002,"[2.147881759341033e-06, 2.594677439115478e-06,...",460141
331121,175.45.176.3,149.171.126.12,0.000022,2.614421e-05,0.023412,0.000298,0.098712,0.000298,0.000013,0.000014,...,0.487988,0.487988,2.936818e-05,2.938642e-05,0.000021,0.000021,0.0,0.000021,"[2.1642257961737377e-05, 2.6144213116305557e-0...",331121
603484,175.45.176.3,149.171.126.19,0.000006,8.373461e-06,0.144476,0.000167,0.002813,0.000067,0.000008,0.000008,...,0.137182,0.137182,1.608996e-06,1.622462e-06,0.000006,0.000006,0.0,0.000006,"[6.08403937818468e-06, 8.373461389523251e-06, ...",603484
2199805,59.166.0.2,149.171.126.5,0.000002,6.363477e-10,0.027949,0.000044,0.001002,0.000019,0.000001,0.000001,...,0.018132,0.031731,3.617282e-07,3.617282e-07,0.000002,0.000002,0.0,0.000002,"[2.274586239684794e-06, 6.363476555465527e-10,...",2199805
730879,175.45.176.1,149.171.126.12,0.000019,2.271349e-05,0.022462,0.000311,0.479822,0.000569,0.000012,0.000012,...,0.423953,0.423953,2.587759e-05,2.587759e-05,0.000018,0.000018,0.0,0.000018,"[1.880229519156918e-05, 2.271349012811647e-05,...",730879


In [14]:
lab_enc = preprocessing.LabelEncoder()
lab_enc.fit(data["Attack"])

# Transform on training set
train["Attack"] = lab_enc.transform(train["Attack"])

# Transform on testing set
test["Attack"] = lab_enc.transform(test["Attack"])

In [15]:
# Training graph (Modified)

train['id'] = train.index

train_g = nx.from_pandas_edgelist(train, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
           ["h", "Label", "Attack", "id"], create_using=nx.MultiGraph())
train_g = train_g.to_directed()
train_g = dgl.from_networkx(train_g, edge_attrs=['h', 'Attack', 'Label', "id"])
nfeat_weight = torch.ones([train_g.number_of_nodes(),
train_g.edata['h'].shape[1]])
train_g.ndata['h'] = nfeat_weight

test['id'] = test.index

# Testing graph
test_g = nx.from_pandas_edgelist(test, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack", "id"], create_using=nx.MultiGraph())
# print(test_g)
test_g = test_g.to_directed()
# print(test_g)
test_g = dgl.from_networkx(test_g, edge_attrs=['h', 'Attack', 'Label', "id"])
nfeat_weight = torch.ones([test_g.number_of_nodes(),
test_g.edata['h'].shape[1]])
test_g.ndata['h'] = nfeat_weight

In [16]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import tqdm
import gc

class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
      super(SAGELayer, self).__init__()
      self.W_apply = nn.Linear(ndim_in + edims , ndim_out)
      self.activation = F.relu
      self.W_edge = nn.Linear(128 * 2, 256)
      self.reset_parameters()

    def reset_parameters(self):
      gain = nn.init.calculate_gain('relu')
      nn.init.xavier_uniform_(self.W_apply.weight, gain=gain)

    def message_func(self, edges):
      return {'m':  edges.data['h']}

    def forward(self, g_dgl, nfeats, efeats):
      with g_dgl.local_scope():
        g = g_dgl
        g.ndata['h'] = nfeats
        g.edata['h'] = efeats
        g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
        g.ndata['h'] = F.relu(self.W_apply(torch.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))

        # Compute edge embeddings
        u, v = g.edges()
        edge = self.W_edge(torch.cat((g.srcdata['h'][u], g.dstdata['h'][v]), 2))
        return g.ndata['h'], edge

In [17]:
class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim,  activation):
      super(SAGE, self).__init__()
      self.layers = nn.ModuleList()
      self.layers.append(SAGELayer(ndim_in, edim, 128, F.relu))

    def forward(self, g, nfeats, efeats, corrupt=False):
      if corrupt:
        e_perm = torch.randperm(g.number_of_edges())
        #n_perm = torch.randperm(g.number_of_nodes())
        efeats = efeats[e_perm]
        #nfeats = nfeats[n_perm]
      for i, layer in enumerate(self.layers):
        #nfeats = layer(g, nfeats, efeats)
        nfeats, e_feats = layer(g, nfeats, efeats)
      #return nfeats.sum(1)
      return nfeats.sum(1), e_feats.sum(1)

In [18]:
class Discriminator(nn.Module):
    def __init__(self, n_hidden):
      super(Discriminator, self).__init__()
      self.weight = nn.Parameter(torch.Tensor(n_hidden, n_hidden))
      self.reset_parameters()

    def uniform(self, size, tensor):
      bound = 1.0 / math.sqrt(size)
      if tensor is not None:
        tensor.data.uniform_(-bound, bound)

    def reset_parameters(self):
      size = self.weight.size(0)
      self.uniform(size, self.weight)

    def forward(self, features, summary):
      features = torch.matmul(features, torch.matmul(self.weight, summary))
      return features

In [19]:
class DGI(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation):
      super(DGI, self).__init__()
      self.encoder = SAGE(ndim_in, ndim_out, edim,  F.relu)
      #self.discriminator = Discriminator(128)
      self.discriminator = Discriminator(256)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)

      positive = positive[1]
      negative = negative[1]

      summary = torch.sigmoid(positive.mean(dim=0))

      positive = self.discriminator(positive, summary)
      negative = self.discriminator(negative, summary)

      l1 = self.loss(positive, torch.ones_like(positive))
      l2 = self.loss(negative, torch.zeros_like(negative))

      return l1 + l2

In [20]:
ndim_in = train_g.ndata['h'].shape[1]
hidden_features = 128
ndim_out = 128
num_layers = 1
edim = train_g.edata['h'].shape[1]
learning_rate = 1e-3
epochs = 4000

In [21]:
dgi = DGI(ndim_in,
    ndim_out,
    edim,
    F.relu)

dgi = dgi.to('cuda')

dgi_optimizer = torch.optim.Adam(dgi.parameters(),
                lr=1e-3,
                weight_decay=0.)

In [22]:
# Format node and edge features for E-GraphSAGE
train_g.ndata['h'] = torch.reshape(train_g.ndata['h'],
                                   (train_g.ndata['h'].shape[0], 1,
                                    train_g.ndata['h'].shape[1]))

train_g.edata['h'] = torch.reshape(train_g.edata['h'],
                                   (train_g.edata['h'].shape[0], 1,
                                    train_g.edata['h'].shape[1]))

In [23]:
# Convert to GPU
train_g = train_g.to('cuda')

In [24]:
cnt_wait = 0
best = 1e9
best_t = 0
dur = []
node_features = train_g.ndata['h'] 
edge_features = train_g.edata['h']

for epoch in range(epochs):
    dgi.train()
    if epoch >= 3:
        t0 = time.time()

    dgi_optimizer.zero_grad()
    loss = dgi(train_g, node_features, edge_features)
    loss.backward()
    dgi_optimizer.step()

    if loss < best:
        best = loss
        best_t = epoch
        cnt_wait = 0
        torch.save(dgi.state_dict(), 'best_dgi_UNSW_v3.pkl')
    else:
        cnt_wait += 1

  # if cnt_wait == patience:
  #     print('Early stopping!')
  #     break

    if epoch >= 3:
        dur.append(time.time() - t0)

    if epoch % 50 == 0:

        print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
            "ETputs(KTEPS) {:.2f}".format(epoch, np.mean(dur),
              loss.item(),
              train_g.num_edges() / np.mean(dur) / 1000))

/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Epoch 00000 | Time(s) nan | Loss 1.6100 | ETputs(KTEPS) nan
Epoch 00050 | Time(s) 0.0994 | Loss 1.2811 | ETputs(KTEPS) 1899.63
Epoch 00100 | Time(s) 0.0973 | Loss 0.2667 | ETputs(KTEPS) 1942.05
Epoch 00150 | Time(s) 0.0967 | Loss 0.0204 | ETputs(KTEPS) 1953.21
Epoch 00200 | Time(s) 0.0963 | Loss 0.0090 | ETputs(KTEPS) 1961.48
Epoch 00250 | Time(s) 0.0959 | Loss 0.0052 | ETputs(KTEPS) 1969.99
Epoch 00300 | Time(s) 0.0955 | Loss 0.0046 | ETputs(KTEPS) 1977.97
Epoch 00350 | Time(s) 0.0952 | Loss 0.0026 | ETputs(KTEPS) 1983.45
Epoch 00400 | Time(s) 0.0950 | Loss 0.0019 | ETputs(KTEPS) 1987.57
Epoch 00450 | Time(s) 0.0949 | Loss 0.0019 | ETputs(KTEPS) 1991.13
Epoch 00500 | Time(s) 0.0947 | Loss 0.0016 | ETputs(KTEPS) 1994.50
Epoch 00550 | Time(s) 0.0946 | Loss 0.0015 | ETputs(KTEPS) 1996.83
Epoch 00600 | Time(s) 0.0945 | Loss 0.0012 | ETputs(KTEPS) 1998.12
Epoch 00650 | Time(s) 0.0945 | Loss 0.0007 | ETputs(KTEPS) 1999.59
Epoch 00700 | Time(s) 0.0944 | Loss 0.0008 | ETputs(KTEPS) 2000.72
Ep

In [25]:
dgi.load_state_dict(torch.load('best_dgi_UNSW_v3.pkl'))

<All keys matched successfully>

In [26]:
training_emb = dgi.encoder(train_g, train_g.ndata['h'], train_g.edata['h'])[1]
training_emb = training_emb.detach().cpu().numpy()

In [27]:
test_g.ndata['h'] = torch.reshape(test_g.ndata['h'],
                                   (test_g.ndata['h'].shape[0], 1,
                                    test_g.ndata['h'].shape[1]))



test_g.edata['h'] = torch.reshape(test_g.edata['h'],
                                   (test_g.edata['h'].shape[0], 1,
                                    test_g.edata['h'].shape[1]))

In [28]:
# Convert to GPU
test_g = test_g.to('cuda')

In [29]:
testing_emb = dgi.encoder(test_g, test_g.ndata['h'], test_g.edata['h'])[1]
testing_emb = testing_emb.detach().cpu().numpy()

In [30]:
# Multimodal (Fusion) Learning

df_train = pd.DataFrame(training_emb,)
# map the id to the original data
df_train['id'] = train_g.edata['id'].detach().cpu().numpy()


df_raw_train = pd.DataFrame(X_train.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h"]))
df_fuse_train = pd.merge(df_train, df_raw_train, on='id', how='left')
df_fuse_train = df_fuse_train.drop(columns=["id"])
df_fuse_train["Attacks"] = train_g.edata['Attack'].detach().cpu().numpy()
df_fuse_train["Label"] = train_g.edata['Label'].detach().cpu().numpy()

df_test = pd.DataFrame(testing_emb,)
# map the id to the original data
df_test['id'] = test_g.edata['id'].detach().cpu().numpy()

df_raw_test = pd.DataFrame(X_test.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h"]))
df_raw_test = pd.merge(df_test, df_raw_test, on='id', how='left')
df_fuse_test = df_raw_test.drop(columns=["id"])
df_fuse_test["Attacks"] = test_g.edata['Attack'].detach().cpu().numpy()
df_fuse_test["Label"] = test_g.edata['Label'].detach().cpu().numpy()

# Embeddings CBLOF  Embeddings

In [31]:
import torch
import dgl
import numpy as np
import pandas as pd
import torch.optim as optim
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import IsolationForest
from pyod.models.cblof import CBLOF
import gc

from tqdm import tqdm
import itertools

In [32]:
benign_fuse_train_samples = df_fuse_train[df_fuse_train.Label == 0].drop(columns=["Label", "Attacks"])
normal_fuse_train_samples = df_fuse_train.drop(columns=["Label", "Attacks"])

fuse_train_labels = df_fuse_train["Label"]
fuse_test_labels = df_fuse_test["Label"]

fuse_test_samples = df_fuse_test.drop(columns=["Label", "Attacks"])

In [33]:
fuse_test_samples

,0,1,2,3,4,5,6,7,8,9,...,NUM_PKTS_512_TO_1024_BYTES,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE
0,0.120290,0.033143,0.199002,0.138511,0.153536,0.230251,0.206177,0.051485,0.173025,-0.019700,...,0.0,0.00000,0.444604,0.444604,0.000024,0.000024,0.000019,0.000019,0.0,0.000019
1,0.120290,0.033143,0.199002,0.138511,0.153536,0.230251,0.206177,0.051485,0.173025,-0.019700,...,0.0,0.00000,0.635577,0.635577,0.000035,0.000035,0.000028,0.000028,0.0,0.000027
2,0.120290,0.033143,0.199002,0.138511,0.153536,0.230251,0.206177,0.051485,0.173025,-0.019700,...,0.0,0.00007,0.573235,0.573235,0.000034,0.000034,0.000025,0.000025,0.0,0.000025
3,0.120290,0.033143,0.199002,0.138511,0.153536,0.230251,0.206177,0.051485,0.173025,-0.019700,...,0.0,0.00000,0.551344,0.551344,0.000024,0.000024,0.000024,0.000024,0.0,0.000024
4,0.120290,0.033143,0.199002,0.138511,0.153536,0.230251,0.206177,0.051485,0.173025,-0.019700,...,0.0,0.00000,0.528801,0.528801,0.000029,0.000029,0.000023,0.000023,0.0,0.000023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80951,0.240966,0.036713,0.211662,0.092804,0.070580,0.213698,0.248727,-0.035184,0.200285,0.023412,...,0.0,0.00000,0.000000,0.000000,0.000002,0.000002,0.000004,0.000004,0.0,0.000004
80952,0.240966,0.036713,0.211662,0.092804,0.070580,0.213698,0.248727,-0.035184,0.200285,0.023412,...,0.0,0.00000,0.000000,0.000000,0.000002,0.000002,0.000004,0.000004,0.0,0.000004
80953,0.241003,0.036675,0.211645,0.092855,0.070612,0.213661,0.248740,-0.035199,0.200298,0.023366,...,0.0,0.00000,0.000000,0.000000,0.000002,0.000002,0.000004,0.000004,0.0,0.000004
80954,0.241003,0.036675,0.211645,0.092855,0.070612,0.213661,0.248740,-0.035199,0.200298,0.023366,...,0.0,0.00000,0.000000,0.000000,0.000002,0.000002,0.000004,0.000004,0.0,0.000004


In [34]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [ ]:
n_est = [5,6,7,9,10] # cant be lower than 5 or 4
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    try:
        clf_if = CBLOF(n_clusters=n_est, contamination=con)
        clf_if.fit(benign_fuse_train_samples)
    except Exception as e:
        print(n_est)
        continue
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

 47%|████▋     | 14/30 [02:41<02:37,  9.86s/it]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fa0da177850>>
Traceback (most recent call last):
  File "/home/kienho/.local/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
 53%|█████▎    | 16/30 [03:03<02:26, 10.45s/it]

In [ ]:
n_est = [5,6,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    try:
        clf_if = CBLOF(n_clusters=n_est, contamination=con)
        clf_if.fit(normal_fuse_train_samples)
    except Exception as e:
        print(n_est)
        continue
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 30/30 [05:13<00:00, 10.44s/it]

{'n_estimators': 5, 'con': 0.2}
0.7246850923456903
              precision    recall  f1-score   support

           0     0.9713    0.8682    0.9168    374702
           1     0.4050    0.7773    0.5325     43236

    accuracy                         0.8588    417938
   macro avg     0.6881    0.8228    0.7247    417938
weighted avg     0.9127    0.8588    0.8771    417938



In [ ]:
# HBOS  Embeddings+Raw (Multimodal)

In [ ]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [ ]:
from pyod.models.hbos import HBOS

n_est = [5,10,15,20,25,30]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [05:45<00:00,  9.60s/it]

{'n_estimators': 25, 'con': 0.05}
0.8471039149382792
              precision    recall  f1-score   support

           0     1.0000    0.9177    0.9571    374702
           1     0.5837    1.0000    0.7371     43236

    accuracy                         0.9262    417938
   macro avg     0.7918    0.9588    0.8471    417938
weighted avg     0.9569    0.9262    0.9343    417938



In [ ]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [06:05<00:00, 10.16s/it]

{'n_estimators': 25, 'con': 0.1}
0.8471039149382792
              precision    recall  f1-score   support

           0     1.0000    0.9177    0.9571    374702
           1     0.5837    1.0000    0.7371     43236

    accuracy                         0.9262    417938
   macro avg     0.7918    0.9588    0.8471    417938
weighted avg     0.9569    0.9262    0.9343    417938



In [ ]:
##  PCA  Emb+Raw (Multimodal/Fusion) Learning

In [ ]:
from pyod.models.pca import PCA
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [05:07<00:00,  8.55s/it]

{'n_estimators': 5, 'con': 0.1}
0.8207887532486517
              precision    recall  f1-score   support

           0     1.0000    0.8987    0.9466    374702
           1     0.5325    1.0000    0.6949     43236

    accuracy                         0.9092    417938
   macro avg     0.7662    0.9493    0.8208    417938
weighted avg     0.9516    0.9092    0.9206    417938



In [ ]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [05:49<00:00,  9.69s/it]

{'n_estimators': 15, 'con': 0.2}
0.8085873162698853
              precision    recall  f1-score   support

           0     1.0000    0.8893    0.9414    374702
           1     0.5103    1.0000    0.6758     43236

    accuracy                         0.9007    417938
   macro avg     0.7552    0.9446    0.8086    417938
weighted avg     0.9493    0.9007    0.9139    417938



In [ ]:
##  IF  Emb+Raw

In [ ]:
for i in range(len(benign_fuse_train_samples.columns)):
    benign_fuse_train_samples.rename(columns={benign_fuse_train_samples.columns[i]: f"feature {i}"}, inplace=True)

for i in range(len(normal_fuse_train_samples.columns)):
    normal_fuse_train_samples.rename(columns={normal_fuse_train_samples.columns[i]: f"feature {i}"}, inplace=True)

for i in range(len(fuse_test_samples.columns)):
    fuse_test_samples.rename(columns={fuse_test_samples.columns[i]: f"feature {i}"}, inplace=True)

In [ ]:
from sklearn.ensemble import IsolationForest
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples.to_numpy())
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

  0%|          | 0/24 [00:00<?, ?it/s]/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
  4%|▍         | 1/24 [00:02<00:54,  2.36s/it]/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
  8%|▊         | 2/24 [00:04<00:51,  2.34s/it]/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
 12%|█▎        | 3/24 [00:07<00:48,  2.33s/it]/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest wa

{'n_estimators': 50, 'con': 0.1}
0.820368662501618
              precision    recall  f1-score   support

           0     1.0000    0.8984    0.9465    374702
           1     0.5317    1.0000    0.6943     43236

    accuracy                         0.9089    417938
   macro avg     0.7659    0.9492    0.8204    417938
weighted avg     0.9516    0.9089    0.9204    417938



In [ ]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 24/24 [01:52<00:00,  4.70s/it]

{'n_estimators': 150, 'con': 0.2}
0.802806141381684
              precision    recall  f1-score   support

           0     0.9991    0.8861    0.9392    374702
           1     0.5015    0.9928    0.6664     43236

    accuracy                         0.8972    417938
   macro avg     0.7503    0.9395    0.8028    417938
weighted avg     0.9476    0.8972    0.9110    417938



## Supervised attack classification (Fusion)

We now train a supervised classifier on the fused features to predict multi-class attack labels:
- Features: embeddings + raw numeric features from `df_fuse_train`/`df_fuse_test` (without `Label`, `Attacks`).
- Target: `Attacks` (encoded integer classes from earlier `LabelEncoder`).
- Model: HistGradientBoostingClassifier (fast, strong on tabular data). Class imbalance handled via per-sample weights.
- Metrics: macro F1, per-class report, and confusion matrix.

In [34]:
# Prepare supervised train/test for attack classification
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, f1_score, confusion_matrix
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.utils import class_weight
import pickle

# Build train features/targets from already prepared fused DataFrames
X_sup_train = df_fuse_train.drop(columns=["Label", "Attacks"]).copy()
y_sup_train = df_fuse_train["Attacks"].copy()

X_sup_test = df_fuse_test.drop(columns=["Label", "Attacks"]).copy()
y_sup_test = df_fuse_test["Attacks"].copy()

# Compute sample weights to mitigate class imbalance on training set
classes = np.unique(y_sup_train)
class_w = class_weight.compute_class_weight(
    class_weight="balanced", classes=classes, y=y_sup_train
)
class_to_w = {c: w for c, w in zip(classes, class_w)}
sample_weight = y_sup_train.map(class_to_w).values

# Feature names to all str
X_sup_train.columns = X_sup_train.columns.map(str)
X_sup_test.columns = X_sup_test.columns.map(str)

## Classification Methods (Fused Features)

Now we'll compare multiple classification algorithms on the fused features:

In [35]:
# Random Forest with Grid Search
from sklearn.ensemble import RandomForestClassifier
import pickle

print("="*60)
print("RANDOM FOREST CLASSIFIER (Fused Features)")
print("="*60)

try:
    n_estimators_grid = [100, 200, 300]
    max_depth_grid = [10, 20, 30]
    params = list(itertools.product(n_estimators_grid, max_depth_grid))

    score = -1
    best_params = None
    best_model = None

    print(f"Testing {len(params)} configurations...")

    for n_est, depth in tqdm(params):
        try:
            rf_clf = RandomForestClassifier(
                n_estimators=n_est,
                max_depth=depth,
                class_weight='balanced',
                random_state=13,
                n_jobs=-1
            )
            
            rf_clf.fit(X_sup_train, y_sup_train)
            y_pred_rf = rf_clf.predict(X_sup_test)
            rf_f1 = f1_score(y_sup_test, y_pred_rf, average='macro')
            
            if rf_f1 > score:
                score = rf_f1
                best_params = {
                    'n_estimators': n_est,
                    'max_depth': depth
                }
                best_model = rf_clf
                # Save best model
                with open('best_rf_classifier_fused.pkl', 'wb') as f:
                    pickle.dump(rf_clf, f)
        except Exception as e:
            print(f"\nError with params (n={n_est}, d={depth}): {str(e)}")
            continue

    if best_model is not None:
        print("\nBest Parameters:", best_params)
        print(f"Best Macro F1: {score:.4f}")
        print("Model saved to: best_rf_classifier_fused.pkl\n")
        print(classification_report(y_sup_test, best_model.predict(X_sup_test), digits=4))
    else:
        print("\nAll configurations failed.")
        
except Exception as e:
    print(f"Unexpected error in Random Forest: {str(e)}")

RANDOM FOREST CLASSIFIER (Fused Features)
Testing 9 configurations...


100%|██████████| 9/9 [03:29<00:00, 23.30s/it]



Best Parameters: {'n_estimators': 300, 'max_depth': 20}
Best Macro F1: 0.5253
Model saved to: best_rf_classifier_fused.pkl

              precision    recall  f1-score   support

           0     0.2864    0.6929    0.4052       736
           1     0.8247    0.1546    0.2604      2070
           2     1.0000    0.9999    1.0000     25812
           3     0.3801    0.2172    0.2764      3030
           4     0.7450    0.7581    0.7515     23290
           5     0.6743    0.8695    0.7596     15354
           6     0.7319    0.3708    0.4922      2856
           7     0.7181    0.5836    0.6439      6774
           8     0.4727    0.3088    0.3736       952
           9     0.2139    0.4512    0.2902        82

    accuracy                         0.7862     80956
   macro avg     0.6047    0.5407    0.5253     80956
weighted avg     0.7906    0.7862    0.7757     80956



In [36]:
# XGBoost with Grid Search
try:
    from xgboost import XGBClassifier
    import pickle
    
    print("="*60)
    print("XGBOOST CLASSIFIER (Fused Features)")
    print("="*60)
    
    n_estimators_grid = [100, 200, 300]
    max_depth_grid = [6, 8, 10]
    learning_rate_grid = [0.01, 0.05, 0.1]
    
    params = list(itertools.product(n_estimators_grid, max_depth_grid, learning_rate_grid))
    
    score = -1
    best_params = None
    best_model = None
    
    print(f"Testing {len(params)} configurations...")
    
    for n_est, depth, lr in tqdm(params):
        try:
            xgb_clf = XGBClassifier(
                n_estimators=n_est,
                max_depth=depth,
                learning_rate=lr,
                random_state=13,
                tree_method='hist', 
                device="cuda"
            )
            
            xgb_clf.fit(X_sup_train, y_sup_train, sample_weight=sample_weight)
            y_pred_xgb = xgb_clf.predict(X_sup_test)
            xgb_f1 = f1_score(y_sup_test, y_pred_xgb, average='macro')
            
            if xgb_f1 > score:
                score = xgb_f1
                best_params = {
                    'n_estimators': n_est,
                    'max_depth': depth,
                    'learning_rate': lr
                }
                best_model = xgb_clf
                # Save best model
                xgb_clf.save_model('best_xgb_classifier_fused.json')
        except Exception as e:
            print(f"\nError with params (n={n_est}, d={depth}, lr={lr}): {str(e)}")
            continue
    
    if best_model is not None:
        print("\nBest Parameters:", best_params)
        print(f"Best Macro F1: {score:.4f}")
        print("Model saved to: best_xgb_classifier_fused.json\n")
        print(classification_report(y_sup_test, best_model.predict(X_sup_test), digits=4))
    else:
        print("\nAll configurations failed. XGBoost might not support your GPU.")
    
except ImportError:
    print("XGBoost not installed. Install with: pip install xgboost")
except Exception as e:
    print(f"Unexpected error: {str(e)}")

XGBOOST CLASSIFIER (Fused Features)
Testing 27 configurations...


  0%|          | 0/27 [00:00<?, ?it/s]/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:37:55] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
100%|██████████| 27/27 [19:16<00:00, 42.83s/it]



Best Parameters: {'n_estimators': 300, 'max_depth': 8, 'learning_rate': 0.1}
Best Macro F1: 0.5647
Model saved to: best_xgb_classifier_fused.json

              precision    recall  f1-score   support

           0     0.2907    0.4946    0.3662       736
           1     0.3924    0.2256    0.2865      2070
           2     0.9590    0.9999    0.9790     25812
           3     0.3018    0.4026    0.3450      3030
           4     0.8416    0.6337    0.7230     23290
           5     0.6984    0.8761    0.7772     15354
           6     0.5502    0.5753    0.5625      2856
           7     0.6730    0.6519    0.6623      6774
           8     0.4403    0.5032    0.4696       952
           9     0.3952    0.5976    0.4757        82

    accuracy                         0.7740     80956
   macro avg     0.5543    0.5961    0.5647     80956
weighted avg     0.7856    0.7740    0.7724     80956



In [37]:
# CatBoost with Grid Search (Expanded, no l2_leaf_reg/border_count, fixed best score logic)
try:
    from catboost import CatBoostClassifier
    
    print("Starting CatBoost Grid Search (Expanded)...")
    
    # Expanded hyperparameter grid
    iterations_grid = [200, 300, 500]
    depth_grid = [4, 8, 10]
    learning_rate_grid = [0.01, 0.05, 0.1]
    
    params = list(itertools.product(iterations_grid, depth_grid, learning_rate_grid))
    print(f"Testing {len(params)} configurations...")
    
    best_score = -1
    best_params = None
    best_model = None
    best_model_path = None
    
    for iterations, depth, lr in tqdm(params):
        try:
            cat_clf = CatBoostClassifier(
                iterations=iterations,
                depth=depth,
                learning_rate=lr,
                auto_class_weights='Balanced',
                random_seed=13,
                verbose=False,
                task_type='GPU'  # Use 'CPU' if GPU not available
            )
            
            cat_clf.fit(X_sup_train, y_sup_train)
            y_pred_cat = cat_clf.predict(X_sup_test)
            cat_f1 = f1_score(y_sup_test, y_pred_cat, average='macro')
            
            if cat_f1 > best_score:
                best_score = cat_f1
                best_params = {
                    'iterations': iterations,
                    'depth': depth,
                    'learning_rate': lr
                }
                best_model = cat_clf
                # Save the best model with unique name for fused features
                best_model_path = "best_catboost_classifier_fused.cbm"
                best_model.save_model(best_model_path)
        except Exception as e:
            print(f"\nError with params (it={iterations}, d={depth}, lr={lr}): {str(e)}")
            continue
    
    print("\n" + "="*60)
    print("BEST CATBOOST HYPERPARAMETERS (Fused Features):")
    print(best_params)
    print(f"Best Macro F1: {best_score:.4f}")
    print("="*60)
    if best_model_path:
        print(f"\nBest CatBoost model saved to: {best_model_path}")
    
    # Final evaluation
    y_pred_best = best_model.predict(X_sup_test)
    print("\n" + "="*60)
    print("FINAL CLASSIFICATION REPORT (Best CatBoost - Fused):")
    print("="*60)
    print(classification_report(y_sup_test, y_pred_best, digits=4))
    
except ImportError:
    print("CatBoost not installed. Install with: pip install catboost")

Starting CatBoost Grid Search (Expanded)...
Testing 27 configurations...


100%|██████████| 27/27 [14:50<00:00, 32.98s/it]


BEST CATBOOST HYPERPARAMETERS (Fused Features):
{'iterations': 500, 'depth': 10, 'learning_rate': 0.1}
Best Macro F1: 0.5445

Best CatBoost model saved to: best_catboost_classifier_fused.cbm

FINAL CLASSIFICATION REPORT (Best CatBoost - Fused):
              precision    recall  f1-score   support

           0     0.2905    0.6658    0.4045       736
           1     0.3632    0.2155    0.2705      2070
           2     1.0000    0.9999    1.0000     25812
           3     0.2759    0.4251    0.3346      3030
           4     0.8403    0.6202    0.7137     23290
           5     0.6961    0.8709    0.7738     15354
           6     0.5030    0.5875    0.5420      2856
           7     0.6874    0.6392    0.6624      6774
           8     0.4241    0.5462    0.4775       952
           9     0.1694    0.6220    0.2663        82

    accuracy                         0.7712     80956
   macro avg     0.5250    0.6192    0.5445     80956
weighted avg     0.7953    0.7712    0.7744     80

In [38]:
# Extra Trees Classifier
from sklearn.ensemble import ExtraTreesClassifier
import pickle

print("="*60)
print("EXTRA TREES CLASSIFIER (Fused Features)")
print("="*60)

try:
    n_estimators_grid = [100, 200, 300]
    max_depth_grid = [10, 20, 30]

    params = list(itertools.product(n_estimators_grid, max_depth_grid))

    score = -1
    best_params = None
    best_model = None

    print(f"Testing {len(params)} configurations...")

    for n_est, depth in tqdm(params):
        try:
            et_clf = ExtraTreesClassifier(
                n_estimators=n_est,
                max_depth=depth,
                class_weight='balanced',
                random_state=13,
                n_jobs=-1
            )
            
            et_clf.fit(X_sup_train, y_sup_train)
            y_pred_et = et_clf.predict(X_sup_test)
            et_f1 = f1_score(y_sup_test, y_pred_et, average='macro')
            
            if et_f1 > score:
                score = et_f1
                best_params = {
                    'n_estimators': n_est,
                    'max_depth': depth
                }
                best_model = et_clf
                # Save best model
                with open('best_et_classifier_fused.pkl', 'wb') as f:
                    pickle.dump(et_clf, f)
        except Exception as e:
            print(f"\nError with params (n={n_est}, d={depth}): {str(e)}")
            continue

    if best_model is not None:
        print("\nBest Parameters:", best_params)
        print(f"Best Macro F1: {score:.4f}")
        print("Model saved to: best_et_classifier_fused.pkl\n")
        print(classification_report(y_sup_test, best_model.predict(X_sup_test), digits=4))
    else:
        print("\nAll configurations failed.")
        
except Exception as e:
    print(f"Unexpected error in Extra Trees: {str(e)}")

EXTRA TREES CLASSIFIER (Fused Features)
Testing 9 configurations...


100%|██████████| 9/9 [01:54<00:00, 12.73s/it]



Best Parameters: {'n_estimators': 200, 'max_depth': 20}
Best Macro F1: 0.4549
Model saved to: best_et_classifier_fused.pkl

              precision    recall  f1-score   support

           0     0.2938    0.5924    0.3928       736
           1     0.5602    0.1932    0.2874      2070
           2     1.0000    0.9999    1.0000     25812
           3     0.2758    0.1525    0.1964      3030
           4     0.7024    0.7274    0.7146     23290
           5     0.6446    0.7480    0.6925     15354
           6     0.6161    0.2034    0.3059      2856
           7     0.5465    0.5650    0.5556      6774
           8     0.2255    0.2784    0.2492       952
           9     0.1070    0.2805    0.1549        82

    accuracy                         0.7440     80956
   macro avg     0.4972    0.4741    0.4549     80956
weighted avg     0.7407    0.7440    0.7344     80956



## Raw Features Classification (Comparison Baseline)

Now we'll train classifiers on **raw features only** (without graph embeddings) to compare the benefit of multimodal fusion:

In [39]:
# Prepare raw features (without embeddings)
# Use only the original numeric features from X_train/X_test

X_raw_train = X_train.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h", "id"]).copy()
y_raw_train = y_train["Attack"].copy()

X_raw_test = X_test.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h", "id"]).copy()
y_raw_test = y_test["Attack"].copy()

# Encode labels
y_raw_train_encoded = lab_enc.transform(y_train["Attack"])
y_raw_test_encoded = lab_enc.transform(y_test["Attack"])

# Compute sample weights
classes_raw = np.unique(y_raw_train_encoded)
class_w_raw = class_weight.compute_class_weight(
    class_weight="balanced", classes=classes_raw, y=y_raw_train_encoded
)
class_to_w_raw = {c: w for c, w in zip(classes_raw, class_w_raw)}
sample_weight_raw = pd.Series(y_raw_train_encoded).map(class_to_w_raw).values

# Feature names to str
X_raw_train.columns = X_raw_train.columns.map(str)
X_raw_test.columns = X_raw_test.columns.map(str)

print(f"Raw feature shape - Train: {X_raw_train.shape}, Test: {X_raw_test.shape}")
print(f"Number of classes: {len(np.unique(y_raw_train_encoded))}")

Raw feature shape - Train: (94447, 39), Test: (40478, 39)
Number of classes: 10


In [40]:
# Raw Features - Random Forest
from sklearn.ensemble import RandomForestClassifier
import pickle

print("="*60)
print("RANDOM FOREST (Raw Features Only)")

n_estimators_grid = [100, 200, 300]
max_depth_grid = [10, 20, 30]
params = list(itertools.product(n_estimators_grid, max_depth_grid))

score = -1
best_params = None
best_model = None

print(f"Testing {len(params)} configurations...")

for n_est, depth in tqdm(params):
    rf_clf = RandomForestClassifier(
        n_estimators=n_est,
        max_depth=depth,
        class_weight='balanced',
        random_state=13,
        n_jobs=-1
    )
    
    rf_clf.fit(X_raw_train, y_raw_train_encoded)
    y_pred = rf_clf.predict(X_raw_test)
    f1 = f1_score(y_raw_test_encoded, y_pred, average='macro')
    
    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est, 'max_depth': depth}
        best_model = rf_clf
        # Save best model
        with open('best_rf_classifier_raw.pkl', 'wb') as f:
            pickle.dump(rf_clf, f)

print("\nBest Parameters:", best_params)
print(f"Best Macro F1: {score:.4f}")
print("Model saved to: best_rf_classifier_raw.pkl\n")
print(classification_report(y_raw_test_encoded, best_model.predict(X_raw_test), digits=4))

RANDOM FOREST (Raw Features Only)
Testing 9 configurations...


100%|██████████| 9/9 [01:51<00:00, 12.38s/it]



Best Parameters: {'n_estimators': 200, 'max_depth': 20}
Best Macro F1: 0.5572
Model saved to: best_rf_classifier_raw.pkl

              precision    recall  f1-score   support

           0     0.2895    0.9457    0.4433       368
           1     0.2491    0.1304    0.1712      1035
           2     0.9997    0.9996    0.9997     12906
           3     0.4437    0.2990    0.3573      1515
           4     0.8183    0.7138    0.7625     11645
           5     0.6760    0.8639    0.7585      7677
           6     0.6817    0.5189    0.5893      1428
           7     0.6685    0.5799    0.6210      3387
           8     0.2902    0.4664    0.3578       476
           9     0.4694    0.5610    0.5111        41

    accuracy                         0.7839     40478
   macro avg     0.5586    0.6079    0.5572     40478
weighted avg     0.7918    0.7839    0.7812     40478



In [41]:
# Raw Features - XGBoost
try:
    from xgboost import XGBClassifier
    
    print("="*60)
    print("XGBOOST (Raw Features Only)")
    print("="*60)
    
    n_estimators_grid = [100, 200, 300]
    max_depth_grid = [6, 8, 10]
    learning_rate_grid = [0.01, 0.05, 0.1]
    
    params = list(itertools.product(n_estimators_grid, max_depth_grid, learning_rate_grid))
    
    score = -1
    best_params = None
    best_model = None
    
    print(f"Testing {len(params)} configurations...")
    
    for n_est, depth, lr in tqdm(params):
        xgb_clf = XGBClassifier(
            n_estimators=n_est,
            max_depth=depth,
            learning_rate=lr,
            random_state=13,
            tree_method='hist',
            device='cuda'
        )
        
        xgb_clf.fit(X_raw_train, y_raw_train_encoded, sample_weight=sample_weight_raw)
        y_pred = xgb_clf.predict(X_raw_test)
        f1 = f1_score(y_raw_test_encoded, y_pred, average='macro')
        
        if f1 > score:
            score = f1
            best_params = {'n_estimators': n_est, 'max_depth': depth, 'learning_rate': lr}
            best_model = xgb_clf
            # Save best model
            xgb_clf.save_model('best_xgb_classifier_raw.json')
    
    print("\nBest Parameters:", best_params)
    print(f"Best Macro F1: {score:.4f}")
    print("Model saved to: best_xgb_classifier_raw.json\n")
    print(classification_report(y_raw_test_encoded, best_model.predict(X_raw_test), digits=4))
    
except ImportError:
    print("XGBoost not installed.")

XGBOOST (Raw Features Only)
Testing 27 configurations...


100%|██████████| 27/27 [04:19<00:00,  9.61s/it]


Best Parameters: {'n_estimators': 300, 'max_depth': 10, 'learning_rate': 0.1}
Best Macro F1: 0.5449
Model saved to: best_xgb_classifier_raw.json

              precision    recall  f1-score   support

           0     0.2954    0.7690    0.4268       368
           1     0.1755    0.2155    0.1934      1035
           2     0.9998    0.9995    0.9997     12906
           3     0.3183    0.3914    0.3511      1515
           4     0.8634    0.6389    0.7344     11645
           5     0.6915    0.8085    0.7455      7677
           6     0.5131    0.5609    0.5360      1428
           7     0.6188    0.6067    0.6127      3387
           8     0.2737    0.5462    0.3647       476
           9     0.4138    0.5854    0.4848        41

    accuracy                         0.7606     40478
   macro avg     0.5163    0.6122    0.5449     40478
weighted avg     0.7909    0.7606    0.7683     40478



In [42]:
# Raw Features - CatBoost (Expanded grid, no l2_leaf_reg/border_count)
try:
    from catboost import CatBoostClassifier
    
    print("="*60)
    print("CATBOOST (Raw Features Only, Expanded)")
    print("="*60)
    
    iterations_grid = [200, 300, 500]
    depth_grid = [4, 8, 10]
    learning_rate_grid = [0.01, 0.05, 0.1]
    
    params = list(itertools.product(iterations_grid, depth_grid, learning_rate_grid))
    
    best_score = -1
    best_params = None
    best_model = None
    best_model_path = None
    
    print(f"Testing {len(params)} configurations...")
    
    for iterations, depth, lr in tqdm(params):
        try:
            cat_clf = CatBoostClassifier(
                iterations=iterations,
                depth=depth,
                learning_rate=lr,
                auto_class_weights='Balanced',
                random_seed=13,
                verbose=False,
                task_type='GPU'
            )
            
            cat_clf.fit(X_raw_train, y_raw_train_encoded)
            y_pred = cat_clf.predict(X_raw_test)
            f1 = f1_score(y_raw_test_encoded, y_pred, average='macro')
            
            if f1 > best_score:
                best_score = f1
                best_params = {'iterations': iterations, 'depth': depth, 'learning_rate': lr}
                best_model = cat_clf
                # Save the best model with unique name for raw features
                best_model_path = "best_catboost_classifier_raw.cbm"
                best_model.save_model(best_model_path)
        except Exception as e:
            print(f"\nError with params (it={iterations}, d={depth}, lr={lr}): {str(e)}")
            continue
    
    print("\nBest Parameters:", best_params)
    print(f"Best Macro F1: {best_score:.4f}\n")
    if best_model_path:
        print(f"Best CatBoost model saved to: {best_model_path}")
    print(classification_report(y_raw_test_encoded, best_model.predict(X_raw_test), digits=4))
    
except ImportError:
    print("CatBoost not installed.")

CATBOOST (Raw Features Only, Expanded)
Testing 27 configurations...


100%|██████████| 27/27 [03:08<00:00,  6.97s/it]


Best Parameters: {'iterations': 500, 'depth': 10, 'learning_rate': 0.1}
Best Macro F1: 0.5070

Best CatBoost model saved to: best_catboost_classifier_raw.cbm
              precision    recall  f1-score   support

           0     0.2764    0.9810    0.4313       368
           1     0.1601    0.2783    0.2032      1035
           2     0.9992    0.9992    0.9992     12906
           3     0.2831    0.4264    0.3403      1515
           4     0.8977    0.5503    0.6823     11645
           5     0.6909    0.7691    0.7279      7677
           6     0.4755    0.5987    0.5301      1428
           7     0.6651    0.5970    0.6292      3387
           8     0.2060    0.6071    0.3076       476
           9     0.1264    0.8049    0.2185        41

    accuracy                         0.7338     40478
   macro avg     0.4781    0.6612    0.5070     40478
weighted avg     0.8001    0.7338    0.7500     40478



In [43]:
# Raw Features - Extra Trees
from sklearn.ensemble import ExtraTreesClassifier
import pickle

print("="*60)
print("EXTRA TREES (Raw Features Only)")
print("="*60)

n_estimators_grid = [100, 200, 300]
max_depth_grid = [10, 20, 30]

params = list(itertools.product(n_estimators_grid, max_depth_grid))

score = -1
best_params = None
best_model = None

print(f"Testing {len(params)} configurations...")

for n_est, depth in tqdm(params):
    et_clf = ExtraTreesClassifier(
        n_estimators=n_est,
        max_depth=depth,
        class_weight='balanced',
        random_state=13,
        n_jobs=-1
    )
    
    et_clf.fit(X_raw_train, y_raw_train_encoded)
    y_pred = et_clf.predict(X_raw_test)
    f1 = f1_score(y_raw_test_encoded, y_pred, average='macro')
    
    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est, 'max_depth': depth}
        best_model = et_clf
        # Save best model
        with open('best_et_classifier_raw.pkl', 'wb') as f:
            pickle.dump(et_clf, f)

print("\nBest Parameters:", best_params)
print(f"Best Macro F1: {score:.4f}")
print("Model saved to: best_et_classifier_raw.pkl\n")
print(classification_report(y_raw_test_encoded, best_model.predict(X_raw_test), digits=4))

EXTRA TREES (Raw Features Only)
Testing 9 configurations...


100%|██████████| 9/9 [00:47<00:00,  5.32s/it]



Best Parameters: {'n_estimators': 300, 'max_depth': 30}
Best Macro F1: 0.5375
Model saved to: best_et_classifier_raw.pkl

              precision    recall  f1-score   support

           0     0.2950    0.8234    0.4344       368
           1     0.2037    0.1266    0.1561      1035
           2     0.9998    0.9995    0.9997     12906
           3     0.4144    0.2667    0.3245      1515
           4     0.7563    0.7490    0.7526     11645
           5     0.6738    0.7667    0.7173      7677
           6     0.6123    0.4867    0.5423      1428
           7     0.6152    0.5621    0.5875      3387
           8     0.3540    0.3004    0.3250       476
           9     0.6333    0.4634    0.5352        41

    accuracy                         0.7685     40478
   macro avg     0.5558    0.5544    0.5375     40478
weighted avg     0.7654    0.7685    0.7640     40478



## Embeddings Only Classification (Graph Features)

Now we'll train classifiers on **embeddings only** (graph features without raw features) to isolate the value of graph-based learning:

In [44]:
# Prepare embeddings-only features (graph features without raw data)
# Extract only the embedding columns (first 256 dimensions from graph encoder)

# From the fused dataframes, extract only embedding columns
# df_fuse_train has: [0-255] = embeddings, [256+] = raw features
num_embedding_dims = 256  # Based on the DGI encoder output dimension

X_emb_train = df_fuse_train.iloc[:, :num_embedding_dims].copy()
y_emb_train = df_fuse_train["Attacks"].copy()

X_emb_test = df_fuse_test.iloc[:, :num_embedding_dims].copy()
y_emb_test = df_fuse_test["Attacks"].copy()

# Compute sample weights
classes_emb = np.unique(y_emb_train)
class_w_emb = class_weight.compute_class_weight(
    class_weight="balanced", classes=classes_emb, y=y_emb_train
)
class_to_w_emb = {c: w for c, w in zip(classes_emb, class_w_emb)}
sample_weight_emb = y_emb_train.map(class_to_w_emb).values

# Feature names to str
X_emb_train.columns = X_emb_train.columns.map(str)
X_emb_test.columns = X_emb_test.columns.map(str)

print(f"Embeddings shape - Train: {X_emb_train.shape}, Test: {X_emb_test.shape}")
print(f"Number of classes: {len(np.unique(y_emb_train))}")
print(f"Using only graph embeddings (no raw features)")

Embeddings shape - Train: (188894, 256), Test: (80956, 256)
Number of classes: 10
Using only graph embeddings (no raw features)


In [45]:
# Embeddings Only - Random Forest
from sklearn.ensemble import RandomForestClassifier
import pickle

print("="*60)
print("RANDOM FOREST (Embeddings Only)")
print("="*60)

n_estimators_grid = [100, 200, 300]
max_depth_grid = [10, 20, 30]
params = list(itertools.product(n_estimators_grid, max_depth_grid))

score = -1
best_params = None
best_model = None

print(f"Testing {len(params)} configurations...")

for n_est, depth in tqdm(params):
    rf_clf = RandomForestClassifier(
        n_estimators=n_est,
        max_depth=depth,
        class_weight='balanced',
        random_state=13,
        n_jobs=-1
    )
    
    rf_clf.fit(X_emb_train, y_emb_train)
    y_pred = rf_clf.predict(X_emb_test)
    f1 = f1_score(y_emb_test, y_pred, average='macro')
    
    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est, 'max_depth': depth}
        best_model = rf_clf
        # Save best model
        with open('best_rf_classifier_embeddings.pkl', 'wb') as f:
            pickle.dump(rf_clf, f)

print("\nBest Parameters:", best_params)
print(f"Best Macro F1: {score:.4f}")
print("Model saved to: best_rf_classifier_embeddings.pkl\n")
print(classification_report(y_emb_test, best_model.predict(X_emb_test), digits=4))

RANDOM FOREST (Embeddings Only)
Testing 9 configurations...


100%|██████████| 9/9 [01:10<00:00,  7.78s/it]



Best Parameters: {'n_estimators': 300, 'max_depth': 30}
Best Macro F1: 0.1977
Model saved to: best_rf_classifier_embeddings.pkl

              precision    recall  f1-score   support

           0     0.0447    0.7215    0.0841       736
           1     0.0738    0.2097    0.1092      2070
           2     1.0000    0.9999    1.0000     25812
           3     0.0870    0.2472    0.1287      3030
           4     0.4311    0.0888    0.1473     23290
           5     0.3434    0.2265    0.2730     15354
           6     0.0903    0.0196    0.0322      2856
           7     0.1500    0.1029    0.1220      6774
           8     0.0464    0.2363    0.0776       952
           9     0.0016    0.0732    0.0032        82

    accuracy                         0.4207     80956
   macro avg     0.2268    0.2926    0.1977     80956
weighted avg     0.5298    0.4207    0.4336     80956



In [46]:
# Embeddings Only - XGBoost
try:
    from xgboost import XGBClassifier
    
    print("="*60)
    print("XGBOOST (Embeddings Only)")
    print("="*60)
    
    n_estimators_grid = [100, 200, 300]
    max_depth_grid = [6, 8, 10]
    learning_rate_grid = [0.01, 0.05, 0.1]
    
    params = list(itertools.product(n_estimators_grid, max_depth_grid, learning_rate_grid))
    
    score = -1
    best_params = None
    best_model = None
    
    print(f"Testing {len(params)} configurations...")
    
    for n_est, depth, lr in tqdm(params):
        xgb_clf = XGBClassifier(
            n_estimators=n_est,
            max_depth=depth,
            learning_rate=lr,
            random_state=13,
            tree_method='hist',
            device='gpu'
        )
        
        xgb_clf.fit(X_emb_train, y_emb_train, sample_weight=sample_weight_emb)
        y_pred = xgb_clf.predict(X_emb_test)
        f1 = f1_score(y_emb_test, y_pred, average='macro')
        
        if f1 > score:
            score = f1
            best_params = {'n_estimators': n_est, 'max_depth': depth, 'learning_rate': lr}
            best_model = xgb_clf
            # Save best model
            xgb_clf.save_model('best_xgb_classifier_embeddings.json')
    
    print("\nBest Parameters:", best_params)
    print(f"Best Macro F1: {score:.4f}")
    print("Model saved to: best_xgb_classifier_embeddings.json\n")
    print(classification_report(y_emb_test, best_model.predict(X_emb_test), digits=4))
    
except ImportError:
    print("XGBoost not installed.")

XGBOOST (Embeddings Only)
Testing 27 configurations...


100%|██████████| 27/27 [11:28<00:00, 25.50s/it]



Best Parameters: {'n_estimators': 300, 'max_depth': 8, 'learning_rate': 0.1}
Best Macro F1: 0.2059
Model saved to: best_xgb_classifier_embeddings.json

              precision    recall  f1-score   support

           0     0.0505    0.4579    0.0910       736
           1     0.0800    0.1783    0.1104      2070
           2     0.9544    0.9999    0.9767     25812
           3     0.0809    0.1888    0.1133      3030
           4     0.4525    0.1630    0.2397     23290
           5     0.3518    0.1820    0.2399     15354
           6     0.0969    0.1250    0.1092      2856
           7     0.1446    0.0977    0.1166      6774
           8     0.0371    0.1639    0.0606       952
           9     0.0009    0.0732    0.0018        82

    accuracy                         0.4306     80956
   macro avg     0.2250    0.2630    0.2059     80956
weighted avg     0.5227    0.4306    0.4481     80956



In [47]:
# Embeddings Only - CatBoost (Expanded grid, no l2_leaf_reg/border_count)
try:
    from catboost import CatBoostClassifier
    
    print("="*60)
    print("CATBOOST (Embeddings Only, Expanded)")
    print("="*60)
    
    iterations_grid = [200, 300, 500]
    depth_grid = [4, 8, 10]
    learning_rate_grid = [0.01, 0.05, 0.1]
    
    params = list(itertools.product(iterations_grid, depth_grid, learning_rate_grid))
    
    best_score = -1
    best_params = None
    best_model = None
    best_model_path = None
    
    print(f"Testing {len(params)} configurations...")
    
    for iterations, depth, lr in tqdm(params):
        try:
            cat_clf = CatBoostClassifier(
                iterations=iterations,
                depth=depth,
                learning_rate=lr,
                auto_class_weights='Balanced',
                random_seed=13,
                verbose=False,
                task_type='GPU'
            )
            
            cat_clf.fit(X_emb_train, y_emb_train)
            y_pred = cat_clf.predict(X_emb_test)
            f1 = f1_score(y_emb_test, y_pred, average='macro')
            
            if f1 > best_score:
                best_score = f1
                best_params = {'iterations': iterations, 'depth': depth, 'learning_rate': lr}
                best_model = cat_clf
                # Save the best model with unique name for embeddings
                best_model_path = "best_catboost_classifier_embeddings.cbm"
                best_model.save_model(best_model_path)
        except Exception as e:
            print(f"\nError with params (it={iterations}, d={depth}, lr={lr}): {str(e)}")
            continue
    
    print("\nBest Parameters:", best_params)
    print(f"Best Macro F1: {best_score:.4f}\n")
    if best_model_path:
        print(f"Best CatBoost model saved to: {best_model_path}")
    print(classification_report(y_emb_test, best_model.predict(X_emb_test), digits=4))
    
except ImportError:
    print("CatBoost not installed.")

CATBOOST (Embeddings Only, Expanded)
Testing 27 configurations...


100%|██████████| 27/27 [09:39<00:00, 21.48s/it]


Best Parameters: {'iterations': 500, 'depth': 10, 'learning_rate': 0.01}
Best Macro F1: 0.2035

Best CatBoost model saved to: best_catboost_classifier_embeddings.cbm
              precision    recall  f1-score   support

           0     0.0503    0.2174    0.0817       736
           1     0.0948    0.1715    0.1221      2070
           2     1.0000    0.9999    1.0000     25812
           3     0.0778    0.2983    0.1234      3030
           4     0.4480    0.1015    0.1656     23290
           5     0.3362    0.2824    0.3070     15354
           6     0.0740    0.0739    0.0739      2856
           7     0.1585    0.0449    0.0699      6774
           8     0.0536    0.2647    0.0891       952
           9     0.0012    0.1341    0.0024        82

    accuracy                         0.4287     80956
   macro avg     0.2294    0.2589    0.2035     80956
weighted avg     0.5338    0.4287    0.4427     80956



In [48]:
# Embeddings Only - Extra Trees
from sklearn.ensemble import ExtraTreesClassifier
import pickle

print("="*60)
print("EXTRA TREES (Embeddings Only)")
print("="*60)

n_estimators_grid = [100, 200, 300]
max_depth_grid = [10, 20, 30]

params = list(itertools.product(n_estimators_grid, max_depth_grid))

score = -1
best_params = None
best_model = None

print(f"Testing {len(params)} configurations...")

for n_est, depth in tqdm(params):
    et_clf = ExtraTreesClassifier(
        n_estimators=n_est,
        max_depth=depth,
        class_weight='balanced',
        random_state=13,
        n_jobs=-1
    )
    
    et_clf.fit(X_emb_train, y_emb_train)
    y_pred = et_clf.predict(X_emb_test)
    f1 = f1_score(y_emb_test, y_pred, average='macro')
    
    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est, 'max_depth': depth}
        best_model = et_clf
        # Save best model
        with open('best_et_classifier_embeddings.pkl', 'wb') as f:
            pickle.dump(et_clf, f)

print("\nBest Parameters:", best_params)
print(f"Best Macro F1: {score:.4f}")
print("Model saved to: best_et_classifier_embeddings.pkl\n")
print(classification_report(y_emb_test, best_model.predict(X_emb_test), digits=4))

EXTRA TREES (Embeddings Only)
Testing 9 configurations...


100%|██████████| 9/9 [00:35<00:00,  3.96s/it]


Best Parameters: {'n_estimators': 100, 'max_depth': 20}
Best Macro F1: 0.1761
Model saved to: best_et_classifier_embeddings.pkl

              precision    recall  f1-score   support

           0     0.0314    0.6495    0.0599       736
           1     0.0864    0.3353    0.1374      2070
           2     1.0000    0.9999    1.0000     25812
           3     0.0839    0.1089    0.0948      3030
           4     0.4025    0.0296    0.0551     23290
           5     0.3489    0.1781    0.2358     15354
           6     0.0758    0.0466    0.0577      2856
           7     0.1295    0.0359    0.0562      6774
           8     0.0357    0.2616    0.0628       952
           9     0.0009    0.0854    0.0018        82

    accuracy                         0.3875     80956
   macro avg     0.2195    0.2731    0.1761     80956
weighted avg     0.5204    0.3875    0.3945     80956



## Summary Comparison Table

Create a comparison table of all methods (Fused vs Raw features):

In [55]:
# Create a summary comparison table
# NOTE: After running all cells above, manually collect the F1 scores and create a comparison

print("="*95)
print("PERFORMANCE COMPARISON: FUSED vs EMBEDDINGS vs RAW FEATURES")
print("="*95)
print("\nInstructions:")
print("1. Run all cells above to get F1 scores for each method")
print("2. Record the best Macro F1 score for each method")
print("3. Compare three approaches:")
print("   - Fused Features: Embeddings + Raw (Multimodal)")
print("   - Embeddings Only: Graph features only")
print("   - Raw Features: Traditional features only")
print("\nExpected format:")
print("-" * 95)
print(f"{'Method':<20} {'Fused (Emb+Raw)':<25} {'Embeddings Only':<25} {'Raw Only':<25}")
print("-" * 95)
print(f"{'Random Forest':<20} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25}")
print(f"{'XGBoost':<20} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25}")
print(f"{'CatBoost':<20} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25}")
print(f"{'Extra Trees':<20} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25}")
print("-" * 95)
print("\nAnalysis:")
print("- Fused features should show best performance (multimodal learning)")
print("- Embeddings capture structural/graph patterns")
print("- Raw features provide traditional statistical information")
print("- Compare to understand the contribution of each modality")

PERFORMANCE COMPARISON: FUSED vs EMBEDDINGS vs RAW FEATURES

Instructions:
1. Run all cells above to get F1 scores for each method
2. Record the best Macro F1 score for each method
3. Compare three approaches:
   - Fused Features: Embeddings + Raw (Multimodal)
   - Embeddings Only: Graph features only
   - Raw Features: Traditional features only

Expected format:
-----------------------------------------------------------------------------------------------
Method               Fused (Emb+Raw)           Embeddings Only           Raw Only                 
-----------------------------------------------------------------------------------------------
Random Forest        [INSERT SCORE]            [INSERT SCORE]            [INSERT SCORE]           
XGBoost              [INSERT SCORE]            [INSERT SCORE]            [INSERT SCORE]           
CatBoost             [INSERT SCORE]            [INSERT SCORE]            [INSERT SCORE]           
Extra Trees          [INSERT SCORE]           

## Key Insights & Interpretation

After running all experiments, analyze the results to answer:

1. **Which approach performs best overall?**
   - Fused features (multimodal) should ideally outperform single modalities
   - Compare the magnitude of improvements

2. **What is the value of graph embeddings?**
   - Compare Embeddings-only vs Raw-only to see if graph structure helps
   - If embeddings alone beat raw features, graph learning is beneficial

3. **Is multimodal fusion effective?**
   - Compare Fused vs (Embeddings + Raw separately)
   - Synergy should provide additional gains beyond individual modalities

4. **Which classifier is most suitable?**
   - Identify the best performing algorithm for each feature type
   - Consider computational cost vs accuracy trade-offs

5. **Feature contribution analysis:**
   - If Fused ≈ Embeddings > Raw: Graph structure dominates
   - If Fused ≈ Raw > Embeddings: Traditional features dominate
   - If Fused > Both: True synergy from multimodal learning